<h1>Basic Training Container example</h1>

This example demonstrates how to run a basic container for training with Amazon SageMaker.

In [ ]:
import boto3
from sagemaker import get_execution_role

prefix = 'basic-training-container'
ecr_repository_name = 'gianpo-ecr/' + prefix
account_id = 825935527263

role = get_execution_role()
region = boto3.Session().region_name
sagemaker_session = sagemaker.session.Session()
bucket = sagemaker_session.default_bucket()

print(account_id)
print(region)
print(role)
print(bucket)

In [ ]:
! ./build_and_push.sh $account_id $region $ecr_repository_name

In [ ]:
container_image_uri = '{0}.dkr.ecr.{1}.amazonaws.com/{2}:latest'.format(account_id, region, ecr_repository_name)
print(container_image_uri)

In [ ]:
sagemaker_session.upload_data('dummy.csv', bucket, prefix + '/train')
sagemaker_session.upload_data('dummy.csv', bucket, prefix + '/val')

In [ ]:
import sagemaker

est = sagemaker.estimator.Estimator(container_image_uri,
                                    role, 
                                    train_instance_count=1, 
                                    train_instance_type='ml.m5.xlarge',
                                    base_job_name=prefix)

est.set_hyperparameters(hp1='value1',
                        hp2=300,
                        hp3=0.001)

train_config = sagemaker.session.s3_input('s3://{0}/{1}/train/'.format(bucket, prefix), content_type='text/csv')
val_config = sagemaker.session.s3_input('s3://{0}/{1}/val/'.format(bucket, prefix), content_type='text/csv')

est.fit({'train': train_config, 'validation': val_config })